## PLX - Phuture Liquid Staking Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import Methodology
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = "liquid-staking-governance-tokens"

# Calculating max slippage based on liquidity threshold
liq = 1.5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = Methodology(min_mcap,
                                min_weight,
                                max_weight,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results, liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    add_category_assets=['benqi','stader','stakewise','stake-dao','yearn-finance','yield-yak'],
    )
liquidity

0x7205705771547cf79201111b4bd8aaf29467b9ec
0x9046d36440290ffde54fe0dd84db8b1cfee9107b
0x9eaf8c1e34f05a589eda6bafdf391cf6ad3cb239
0xaeaeed23478c3a4b798e4ed40d8b7f41366ae861
0xdf474b7109b73b7d57926d43598d5934131136b2
0xaeaeed23478c3a4b798e4ed40d8b7f41366ae861
0x84c882a4d8eb448ce086ea19418ca0f32f106117
Dropping stader because a price feed is unavailable
            lido-dao  frax-share  rocket-pool  yearn-finance     benqi  \
date                                                                     
2023-12-06  0.567862    0.174316     0.163443       0.075505  0.009621   

            stakewise  yield-yak  
date                              
2023-12-06   0.007611   0.001642  


,spot price,delivery price,slippage,blockchain
id,,,,
lido-dao,0.426377,0.425585,-0.001859,ethereum
rocket-pool,0.032990,0.032774,-0.006542,ethereum
frax-share,0.119919,0.117303,-0.021818,ethereum
stakewise,11.609285,11.143220,-0.040146,ethereum
yearn-finance,0.000118,0.000113,-0.045904,ethereum
stader,1.589422,1.479443,-0.069194,ethereum
benqi,104.050740,94.975686,-0.087218,avalanche
yield-yak,0.001482,0.001316,-0.111819,avalanche
stake-dao,2.704972,2.126182,-0.213973,ethereum


In [4]:
results

,name,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
lido-dao,Lido DAO,2.075695e+09,0.300800,77,0x5a98fcbea516cf06857215779fd812ca3bef1b32,ethereum
frax-share,Frax Share,6.296025e+08,0.283120,73,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,ethereum
rocket-pool,Rocket Pool,5.972710e+08,0.265460,68,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,ethereum
yearn-finance,yearn.finance,2.856533e+08,0.122633,31,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,ethereum
benqi,BENQI,4.059664e+07,0.015626,3,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5,avalanche
stakewise,StakeWise,2.813701e+07,0.012361,3,0x48c3399719b582dd63eb5aadf12a40b4c3f52fa2,ethereum
